In [ ]:
from mainnet_launch.data_fetching.quotes.top_level_check_exit_liqudity import main
import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.templates.default = None

quote_df = await main()

2025-07-02 12:06:27.074 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


Index(['aggregatorName', 'internal', 'asyncSwapper', 'minBuyAmount',
       'buyAmount', 'expiration', 'tx', 'fullQuoteDetails', 'chainId',
       'systemName', 'slippageBps', 'taker', 'sellToken', 'buyToken',
       'sellAmount', 'includeSources', 'excludeSources', 'sellAll',
       'timeoutMS', 'transferToCaller', 'same_token', 'token_address',
       'chain_id', 'symbol', 'name', 'decimals', 'buy_amount_norm',
       'min_buy_amount_norm', 'sell_amount_norm', 'ratio'],
      dtype='object')

In [11]:
def compute_excess_slippage_from_size(quote_df:pd.DataFrame) -> pd.DataFrame:
    df = quote_df.groupby(["symbol", "sell_amount_norm"])["ratio"].first().reset_index()
    buy_token_ratio_at_sell_1_token = (
        df[df["sell_amount_norm"] == 1][["symbol", "ratio"]].set_index("symbol").to_dict()["ratio"]
    )
    highest_sold_amount = df.groupby("symbol")["sell_amount_norm"].max().to_dict()

    df["buy_token_ratio_at_one_token_sold"] = df["symbol"].map(buy_token_ratio_at_sell_1_token)
    df["highest_sold_amount"] = df["symbol"].map(highest_sold_amount)
    df["percent_sold"] = df.apply(lambda row: 100 * row["sell_amount_norm"] / row["highest_sold_amount"], axis=1)
    df["bps_excess_loss_vs_1"] = df.apply(
        lambda row: 10_000 * (row["buy_token_ratio_at_one_token_sold"] - row["ratio"]) / row["ratio"], axis=1
    )
    return df

df = compute_excess_slippage_from_size(quote_df)

px.scatter(df, x="percent_sold", y="bps_excess_loss_vs_1", color="symbol").show()
px.scatter(df, x="sell_amount_norm", y="bps_excess_loss_vs_1", color="symbol").show()